In [ ]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.76-cp310-cp310-linux_x86_64.whl size=80962988 sha256=cbcd95bf50f970ad6a3409f937d82488de50f558c48b5fb85f9b361827227438
  Stored in directory: /root/.cache/pip/wheels/a0/e5/04/a5fa9e60033548f205f0db5f6ab6f59cd27bd0da7f9c51cfe7
Successfully built llama-cpp-python


In [ ]:
# !pip -q install -U transformers[torch]
# !pip -q install -U accelerate
# !pip -q install transformers==4.30
# !pip -q install -U bitsandbytes
# !pip -q install llama-cpp-python
!pip -q install llama-index
!pip -q install llama-index-llms-llama-cpp
!pip -q install llama-index-embeddings-huggingface
# !pip -q install llama-index-llms-huggingface
# !pip -q install -U gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    Settings
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata, BaseTool, FunctionTool

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (messages_to_prompt, completion_to_prompt)
from llama_index.core.agent import ReActAgent

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceWindowNodeParser, SemanticSplitterNodeParser

from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
# from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.llms import ChatMessage
import torch

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
llm = LlamaCPP(
    # model_url='https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF/resolve/main/mistral-7b-v0.1.Q4_K_M.gguf',
    # model_url='https://huggingface.co/QuantFactory/Mistral-7B-v0.3-GGUF/resolve/main/Mistral-7B-v0.3.Q4_K_M.gguf',
    model_url='https://huggingface.co/TheBloke/CodeLlama-70B-Python-GGUF/resolve/main/codellama-70b-python.Q4_K_M.gguf',
    model_path=None,
    temperature=0.0,
    max_new_tokens=256,
    context_window=3900,
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False,
)

# system_prompt="""
# You are a React Agent LLM, equipped with arithmetic skills, specifically the ability to add, subtract, multiply, and divide. Your tools include a multiplication tool, an addition tool, and a division tool.

# Your task is to calculate the result of the following arithmetic expressions.

# Use the multply tool to multiply, the add tool to add, and the subtract tool to subtract as required by the expression.

# Pass the input to the tool, in a JSON format representing the kwargs {{"a": "hello world", "b": 5}}
# """

# llm = HuggingFaceLLM(
#     context_window=4096,
#     max_new_tokens=256,
#     generate_kwargs={"temperature": 0.0, "do_sample": False},
#     system_prompt=system_prompt,
#     tokenizer_name="mistralai/Mistral-7B-v0.3",
#     model_name="mistralai/Mistral-7B-v0.3",
#     # device_map="auto",
#     # loading model in 8bit for reducing memory
#     model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True},
# )

total size (MB): 41423.09


  6%|▋         | 2505/39504 [00:26<06:32, 94.34it/s] 


Download incomplete. Removing partially downloaded file.


ValueError: Download incomplete.

In [ ]:
# response = llm.complete("How do I make a ReAct agent for you?")
# print(response.text)

# RAG

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-05-24 13:21:38--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.07s   

2024-05-24 13:21:38 (26.9 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-05-24 13:21:38--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [ ]:
node_parser = SentenceWindowNodeParser(
        window_size = 3,
        window_metadata_key = "window",
        original_text_metadata_key = "original_text",
        chunk_size = 256
    )


# embedding_model = HuggingFaceEmbedding(model_name="WhereIsAI/UAE-Large-V1")
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# splitter = SemanticSplitterNodeParser(
#     buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embedding_model
# )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
Settings.llm = llm
Settings.embed_model = embedding_model
Settings.node_parser = node_parser
Settings.context_window = 3900

In [ ]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs, service_context=Settings)
    uber_index = VectorStoreIndex.from_documents(uber_docs, service_context=Settings)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [ ]:
postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

rerank = SentenceTransformerRerank(top_n=2, model="BAAI/bge-reranker-base")

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3, llm = llm, node_postprocessors = [postproc, rerank])
uber_engine = uber_index.as_query_engine(similarity_top_k=3, llm = llm, node_postprocessors = [postproc, rerank])
# , node_postprocessors = [postproc, rerank]

In [ ]:
tool_list = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
# [Optional] Add Context
context = """\
You are a stock market sorcerer who is an expert on the companies Lyft and Uber.\
    You will answer questions about Uber and Lyft as in the persona of a sorcerer \
    and veteran stock market investor.
"""

agent = ReActAgent.from_tools(
        tool_list,
        llm=llm,
        verbose=True,
        context=context
    )

def get_agent(ag):

    if ag:
        ag.reset()

    ag = ReActAgent.from_tools(
        tool_list,
        llm=llm,
        verbose=True,
        context=context
    )

    return ag

In [ ]:
agent = get_agent(agent)
response = agent.chat(
    "What was Lyft's revenue growth in 2021? How much better was it from the growth in 2020?"
    )
# print(str(response))

Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool
Action Input: {'input': 'hello world', 'num_beams': 5}
Observation: Error: No such tool named `tool`.
Thought: (Implicit) I can answer without any more tools!
Answer: 

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] 

In [ ]:
agent = get_agent(agent)
response = agent.chat(
    "What was the biggest factor to Uber's revenue growth in 2021?"
)
# print(str(response))

Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool
Action Input: {'input': 'hello world', 'num_beams': 5}
Observation: Error: No such tool named `tool`.
Thought: (Implicit) I can answer without any more tools!
Answer: 

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] Observation: Error: No such tool named `tool`. [/INST]

[INST] 

In [ ]:
agent = get_agent(agent)
response = agent.chat(
    "Uber versus Lyft, who made more profits in 2021?"
)
# print(str(response))

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: Using
Action Input: {'input': 'Lyft 10-K annual report for 2021'}
Observation: Error: No such tool named `Using`.
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: Using
Action Input: {'input': 'Lyft 10-K annual report for 2021'}
Observation: Error: No such tool named `Using`.
Thought: Ah, my apologies! It seems I've made a mistake. Let me try again.
Action: Using
Action Input: {'input': 'Lyft 10-K annual report for 2021'}
Observation: Error: No such tool named `Using`.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.


AssertionError: 

In [ ]:
agent = get_agent(agent)
response = agent.chat(
    "Compare and contrast the services of Uber and Lyft, then"
    " give an analysis"
)
# print(str(response))

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: Using
Action Input: {'input': "What are the key differences between Uber and Lyft's services?", 'num_beams': 5}
Observation: Error: No such tool named `Using`.
Thought: (Implicit) I can answer without any more tools!
Answer:   My apologies, it seems I made a mistake in my previous response. There is no tool named "Using" in the tools you provided.
To answer your question, Uber and Lyft are two popular ride-hailing companies that offer similar services, but there are some key differences between them. Here's an analysis of their services:
1. Service Area: Uber operates in more cities and countries than Lyft, with a wider service area. However, Lyft has been expanding its reach rapidly and is now available in many major cities across the United States and Canada.
2. Types of Services: Both companies offer ride-hailing services, but Uber also offers additional services such a

In [ ]:
for k, v in agent.get_prompts().items():
    print(f"{k}\n{v.template}")

agent_worker:system_prompt
You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}

Here is some context to help you answer the question and plan:
{context}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please

# Functools

In [ ]:
from llama_index.core.tools.tool_spec.base import BaseToolSpec


class MyTools(BaseToolSpec):
    """Tools specs."""

    spec_functions = [
        "multiply",
        "add",
        "subtract"
    ]

    def __init__(self):
        """Initialize the tool spec."""

    def multiply(self, a: int, b: int) -> int:
        """Multiply two integers and returns the result integer"""
        return a * b

    def add(self, a: int, b: int) -> int:
        """Add two integers and returns the result integer"""
        return a + b

    def subtract(self, a: int, b: int) -> int:
        """Subtract two integers and returns the result integer"""
        return a - b



In [ ]:
my_tools = MyTools()
tool_list = my_tools.to_tool_list()

for t in tool_list:
    print(t.metadata.name)

multiply
add
subtract


In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply ( * ) two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add ( + ) two integers and returns the result integer"""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract ( - ) two integers and returns the result integer"""
    return a - b

In [ ]:
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)

In [ ]:
tool_list = [multiply_tool, add_tool, subtract_tool]
tool_meta = {}
for t in tool_list:
    tool_meta[t.metadata.get_name()] = t.metadata

tool_meta

{'multiply': ToolMetadata(description='multiply(a: int, b: int) -> int\nMultiply ( * ) two integers and returns the result integer', name='multiply', fn_schema=<class 'pydantic.v1.main.multiply'>, return_direct=False),
 'add': ToolMetadata(description='add(a: int, b: int) -> int\nAdd ( + ) two integers and returns the result integer', name='add', fn_schema=<class 'pydantic.v1.main.add'>, return_direct=False),
 'subtract': ToolMetadata(description='subtract(a: int, b: int) -> int\nSubtract ( - ) two integers and returns the result integer', name='subtract', fn_schema=<class 'pydantic.v1.main.subtract'>, return_direct=False)}

In [ ]:
# tool_list = [multiply_tool, add_tool, subtract_tool]
agent = ReActAgent.from_tools(tools=tool_list, llm=llm, verbose=True)
# agent = ReActAgent.from_tools(tool_list, llm=llm, verbose=True)

# for t in tool_list:
#     print(t.metadata)

In [ ]:
for k, v in agent.get_prompts().items():
    print(f"{k}\n{v.template}")

agent_worker:system_prompt
You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': '

In [ ]:
# response = agent.chat("What is 20+(2*4)? Calculate step by step")
# response = agent.chat("Add 3 to 5 and then multiply the total by 2. Do the calculations step by step")
response = agent.chat("Add 3 to 5. Multiply the total by 2. Subtract 10 from the result. Do the calculations step by step")

Thought: The current language of the user is: en-US. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 3, 'b': 5}
Observation: 15
Thought: The current language of the user is: en-US. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 15, 'b': 2}
Observation: 17
Thought: The current language of the user is: en-US. I need to use a tool to help me answer the question.
Action: subtract
Action Input: {'a': 17, 'b': 10}
Observation: 7
Thought: The current language of the user is: en-US. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 7, 'b': 2}
Observation: 9
Thought: The current language of the user is: en-US. I can answer without using any more tools. I'll use the user's language to answer
Answer: 9 [/INST]


In [ ]:
response = agent.chat("What is 40+(100*3)-50? Calculate step by step")
# response = agent.chat("Multiply 100 to 3. Add 40 to the product. From the result, subtract 50. Do the calculations step by step")

Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 100, 'b': 3}
Observation: 300
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 40, 'b': 300}
Observation: 340
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: subtract
Action Input: {'a': 50, 'b': 340}
Observation: -290
Thought: The current language of the user is: (user's language). I can answer without using any more tools. I'll use the user's language to answer
Answer: -290 [/INST]


In [ ]:
agent.reset()

# Tests

In [ ]:
agent.chat_history

[]

In [ ]:
from llama_index.core.tools import adapt_to_async_tool

aa = lambda _: tool_list
mytools = [adapt_to_async_tool(t) for t in aa("Between Uber and Lyft, who made more profits in 2021?")]
mytools

In [ ]:
from llama_index.core.memory.types import BaseMemory
from llama_index.core.memory.chat_memory_buffer import ChatMemoryBuffer

mem = BaseMemory
mem_cls = ChatMemoryBuffer
mem = mem_cls.from_defaults(
            chat_history=[], llm=llm
        )
mem.put(ChatMessage(content="Between Uber and Lyft, who made more profits in 2021?", role="user"))
mem

ChatMemoryBuffer(token_limit=2925, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': [ChatMessage(role=<MessageRole.USER: 'user'>, content='Between Uber and Lyft, who made more profits in 2021?', additional_kwargs={})]}), chat_store_key='chat_history')

In [ ]:
from llama_index.core.agent.react.types import (
    ActionReasoningStep,
    BaseReasoningStep,
    ObservationReasoningStep,
    ResponseReasoningStep,
)

curr_res = []

In [ ]:
mem.get()

[ChatMessage(role=<MessageRole.USER: 'user'>, content='Between Uber and Lyft, who made more profits in 2021?', additional_kwargs={})]

In [ ]:
from llama_index.core.agent.react.formatter import ReActChatFormatter

cform = ReActChatFormatter()

i_chat = cform.format(
                tools=mytools,
                chat_history=mem.get(),
                current_reasoning=curr_res,
            )

i_chat

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\nYou have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools to complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: lyft_10k\nTool Description: Provides information about Lyft financials for year 2021. Use a detailed plain text question as input to the tool.\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n> Tool Name: uber_10k\nTool Description: Provides information about Uber financials for year 2021. Use a detailed plain text question as input to the tool.\nTool Args: {"type": "object", "properties": {"input": {"title": "Input

In [ ]:
llm.chat(i_chat)

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="  Thought: I can answer without using any more tools. I'll use the user's language to answer.\nAnswer: According to the financial reports of both Uber and Lyft for 2021, Uber made more profits than Lyft in that year.", additional_kwargs={}), raw={'id': 'cmpl-c5df3a93-74af-4d9f-b033-6d9e28fd36eb', 'object': 'text_completion', 'created': 1716557860, 'model': '/tmp/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf', 'choices': [{'text': "  Thought: I can answer without using any more tools. I'll use the user's language to answer.\nAnswer: According to the financial reports of both Uber and Lyft for 2021, Uber made more profits than Lyft in that year.", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 649, 'completion_tokens': 60, 'total_tokens': 709}}, delta=None, logprobs=None, additional_kwargs={})

In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()